In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
import cv2
import numpy as np
import glob
import os
from random import shuffle

In [2]:
from tensorflow import keras

In [3]:
import tensorflow as tf
print(tf.test.gpu_device_name())

In [4]:
X_train=[]
X_test=[]
Y_test=[]
Y_train=[]
dataTr=[]
for filename in glob.glob(os.path.join('D:/UMSA/TESIS/Tesis de grado/data/entrenamiento/Melanoma_escalado','*.jpg')):
    dataTr.append([1,cv2.imread(filename)])
for filename in glob.glob(os.path.join('D:/UMSA/TESIS/Tesis de grado/data/entrenamiento/Carcinoma_escalado','*.jpg')):
    dataTr.append([0,cv2.imread(filename)])

In [5]:
def particion(datos):
    imagenes=[]
    etiquetas=[]
    for i,j in datos:
        imagenes.append(j)
        etiquetas.append(i)
    imagenes=np.array(imagenes)
    etiquetas=np.array(etiquetas)
    return (imagenes,etiquetas)

In [6]:
shuffle(dataTr)
print("en total tenemos: "+str(len(dataTr))+ " imagenes dentro de la carpeta train")

en total tenemos: 13319 imagenes dentro de la carpeta train


In [7]:
porcion1=dataTr[0:5250]
porcion2=dataTr[5251:7501]
print("para entrenamiento tendremos: "+str(len(porcion1))+ " imagenes de la carpeta de train")
print("para prueba tendremos: "+str(len(porcion2))+ " imagenes de la carpeta de train")

para entrenamiento tendremos: 5250 imagenes de la carpeta de train
para prueba tendremos: 2250 imagenes de la carpeta de train


In [8]:
x_e,y_e=particion(porcion1)
x_p,y_p=particion(porcion2)

In [9]:
def validacion(x_e,y_e,x_p,y_p,modelo1,epocas):
    entre=modelo1.fit(x_e,y_e,batch_size=32,epochs=epocas,validation_data=(x_p,y_p))
    a=modelo1.evaluate(x_p,y_p)
    return a[1]

In [10]:
def evaluacion(modelo1,porcentaje,nombre,v1,v2):
    prediccion=0
    epocas=1
    while(True):
        if prediccion < porcentaje:
            prediccion=(validacion(x_e,y_e,x_p,y_p,modelo1,1))*100
            epocas +=1
            v1.append(epocas-1)
            v2.append(prediccion)
        else:
            print("==> Para el metodo "+nombre+" se utilizo: "+str(epocas-1)+" epocas para llegar a mas del "+str(porcentaje)+"% de acertividad")
            break 

In [11]:
modelo=Sequential()
modelo.add(Convolution2D(32,(3,3),input_shape=(224,224,3),activation='relu'))
modelo.add(MaxPooling2D(pool_size=((2,2))))
modelo.add(Flatten())
modelo.add(Dense(128,activation='relu'))
modelo.add(Dense(50,activation='relu'))
modelo.add(Dense(1,activation='sigmoid'))
modelo.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
flatten (Flatten)            (None, 394272)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               50466944  
_________________________________________________________________
dense_1 (Dense)              (None, 50)                6450      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 50,474,341
Trainable params

In [12]:
modelo_adam = keras.optimizers.Adam(learning_rate=0.0008)
modelo.compile(optimizer=modelo_adam,loss='binary_crossentropy',metrics=['accuracy'])
v_eA=[]
v_aA=[]
evaluacion(modelo,80,"ADAM",v_eA,v_aA)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 5250 samples, validate on 2250 samples
2250/2250 [==============================] - 4s 2ms/sample - loss: 2.9092 - acc: 0.6969
Train on 5250 samples, validate on 2250 samples
2250/2250 [==============================] - 4s 2ms/sample - loss: 1.0208 - acc: 0.7307
Train on 5250 samples, validate on 2250 samples
2250/2250 [==============================] - 4s 2ms/sample - loss: 0.8992 - acc: 0.7364
Train on 5250 samples, validate on 2250 samples
2250/2250 [==============================] - 4s 2ms/sample - loss: 1.2003 - acc: 0.6956
Train on 5250 samples, validate on 2250 samples
2250/2250 [==============================] - 4s 2ms/sample - loss: 1.0065 - acc: 0.7742
Train on 5250 samples, validate on 2250 samples
2250/2250 [==============================] - 4s 2ms/sample - loss: 1.0673 - acc: 0.7511
Train on 5250 samples, validate on 2250 samples
2250/2250 [==============================]

In [13]:
modelo.save("D:/UMSA/Documentos/CNN cancer de piel/MODELOS/7500/modelo")

In [14]:
modelo.save_weights("D:/UMSA/Documentos/CNN cancer de piel/PESOS/7500/modelo_pesos")